In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, matthews_corrcoef, recall_score, precision_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import roc_curve
import torch.nn.functional as F
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
loaded_datasets_info = torch.load('/root/autodl-tmp/data/saved_datasets.pth', weights_only=False)
train_dataset = loaded_datasets_info['train_dataset']
val_dataset = loaded_datasets_info['val_dataset']

In [ ]:
from torch.utils.data import DataLoader

def extract_features_labels_from_subset(subset):
    
    loader = DataLoader(subset, batch_size=len(subset))
    
    for features, labels in loader:
        features = features.squeeze(1).numpy()
        labels = labels.squeeze(1).numpy()
        return features, labels

X_train, y_train = extract_features_labels_from_subset(train_dataset)
X_val, y_val = extract_features_labels_from_subset(val_dataset)

In [ ]:
unique, counts = torch.unique(torch.argmax(torch.tensor(y_train), dim=-1), return_counts=True)

for u, c in zip(unique, counts):
    print(f"Class: {u.item()}, Count: {c.item()}")

In [ ]:
unique, counts = torch.unique(torch.argmax(torch.tensor(y_val), dim=-1), return_counts=True)

for u, c in zip(unique, counts):
    print(f"Class: {u.item()}, Count: {c.item()}")

#### Metrics function definition

In [ ]:
# get roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC
def calculate_multiclass_metrics(true_labels, predicted_labels, predicted_probabilities, num_classes):
    accuracy = accuracy_score(true_labels, predicted_labels)
    mcc = matthews_corrcoef(true_labels, predicted_labels)
    
    sensitivity_per_class = []
    specificity_per_class = []
    auc_per_class = []
    f1_per_class = []

    for i in range(num_classes):
        true_binary = (np.array(true_labels) == i).astype(int)
        pred_binary = (np.array(predicted_labels) == i).astype(int)

        cm = confusion_matrix(true_binary, pred_binary, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()

        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity_per_class.append(sensitivity)
        specificity_per_class.append(specificity)

        
        auc = roc_auc_score(true_binary, predicted_probabilities[:, i]) if len(np.unique(true_binary)) > 1 else 0
        auc_per_class.append(auc)

        f1 = f1_score(true_binary, pred_binary) if len(np.unique(true_binary)) > 1 else 0
        f1_per_class.append(f1)

    avg_sensitivity = np.mean(sensitivity_per_class)
    avg_specificity = np.mean(specificity_per_class)
    avg_auc = np.mean(auc_per_class) if auc_per_class else 0
    avg_f1 = np.mean(f1_per_class)

    print(f"Average AUC: {avg_auc:.4f}")
    print(f"Average Sensitivity: {avg_sensitivity:.4f}")
    print(f"Average Specificity: {avg_specificity:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Average F1-score: {avg_f1:.4f}")
    print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")
    
    # Generate classification report
    report = classification_report(true_labels, predicted_labels, target_names=[f'Class {i}' for i in range(num_classes)], digits=4)
    print("\nClassification Report:")
    print(report)

#### XGB

In [ ]:
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',  
    learning_rate=0.1,
    max_depth=10,
    n_estimators=30)

xgb_model.fit(X_train, y_train)

In [ ]:
y_pred = xgb_model.predict(X_train)

In [ ]:
y_pred

#### Training data metrics

In [ ]:
true_labels_ = np.argmax(y_train, axis=-1)
predicted_labels = np.argmax(y_pred, axis=-1)
preds = torch.tensor(y_pred)
preds = F.softmax(preds, dim=-1)

train_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=3)

In [ ]:
np.save('/root/autodl-tmp/ROC/XGB/y_train_pred.npy', preds)
np.save('/root/autodl-tmp/ROC/XGB/y_train.npy', y_train)

#### Validation data metrics

In [ ]:
y_pred = xgb_model.predict(X_val)

In [ ]:
true_labels_ = np.argmax(y_val, axis=-1)
predicted_labels = np.argmax(y_pred, axis=-1)
preds = torch.tensor(y_pred)
preds = F.softmax(preds, dim=-1)

test_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=3)

In [ ]:
np.save('/root/autodl-tmp/ROC/XGB/y_val_pred.npy', preds)
np.save('/root/autodl-tmp/ROC/XGB/y_val.npy', y_val)